In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
idx = pd.IndexSlice

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression

from datetime import timedelta

%aimport trans.data
%aimport trans.gtrans
%aimport trans.reg

from trans.data import GetData
gd = GetData()
from trans.gtrans import *
from trans.reg import Reg


pctTrans     = DataFrameFunctionTransformer(func = lambda s: s.pct_change())
rankTrans    = DataFrameFunctionTransformer(func = lambda s: s.rank(method="first"), axis=1)
pctOnlyTrans = GenSelectAttrsTransformer(['Pct'], dropSingle=False )

## Get the raw data

In [13]:
%aimport trans.data
raw_df = gd.combine_data(['FB', 'AAPL', 'AMZN', 
                           'NFLX', 'GOOG', 'SPY'])
raw_df.head()

Open                                          High            \
             FB      AAPL     AMZN NFLX GOOG         SPY   FB      AAPL   
Date                                                                      
2000-01-03  NaN  3.745536  81.5000  NaN  NaN  148.250000  NaN  4.017857   
2000-01-04  NaN  3.866071  85.3750  NaN  NaN  143.531204  NaN  3.950893   
2000-01-05  NaN  3.705357  70.5000  NaN  NaN  139.937500  NaN  3.948661   
2000-01-06  NaN  3.790179  71.3125  NaN  NaN  139.625000  NaN  3.821429   
2000-01-07  NaN  3.446429  67.0000  NaN  NaN  140.312500  NaN  3.607143   

                             ...     Adj Close                       Volume  \
               AMZN NFLX     ...          AMZN NFLX GOOG         SPY     FB   
Date                         ...                                              
2000-01-03  89.5625  NaN     ...       89.3750  NaN  NaN  103.807343    NaN   
2000-01-04  91.5000  NaN     ...       81.9375  NaN  NaN   99.747833    NaN   
2000-01-05  75.1250  NaN     ...       69.7500  NaN  NaN   99.926254    NaN   
2000-01-06  72.6875  NaN     ...       65.5625  NaN  NaN   98.320305    NaN   
2000-01-07  70.5000  NaN     ...       69.5625  NaN  NaN  104.030365    NaN   

                                                         
                 AAPL        AMZN NFLX GOOG         SPY  
Date                                                     
2000-01-03  133949200  16117600.0  NaN  NaN   8164300.0  
2000-01-04  128094400  17487400.0  NaN  NaN   8089800.0  
2000-01-05  194580400  38457400.0  NaN  NaN  12177900.0  
2000-01-06  191993200  18752000.0  NaN  NaN   6227200.0  
2000-01-07  115183600  10505400.0  NaN  NaN   8066500.0  

[5 rows x 36 columns]

## Define featUn transformer: compute Pct and append to Adj Close

In [14]:
pipe_close = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True )
                      )   

pipe_pct   = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True ), 
                         pctTrans,
                      )

featUn = GenDataFrameFeatureUnion( [ ("Adj Close", pipe_close),
                                    ("Pct", pipe_pct)
                                   ] )

In [15]:
pipe_pct_only   = make_pipeline(GenSelectAttrsTransformer(['Pct'], dropSingle=False ) ) 

## Create pipeline to prepare data for regression

In [16]:
pipe_nn = make_pipeline( featUn,
                         DatetimeIndexTransformer("Dt"),
                         pctOnlyTrans,
                         RestrictToCalendarColTransformer( ("Pct", "SPY")),
                         RestrictToNonNullTransformer("all"),
                         # FillNullTransformer(method="bfill")
                       )
pct_df = pipe_nn.fit_transform(raw_df)
pct_df.head()

Pct                                  
                AAPL      AMZN  FB GOOG NFLX       SPY
Dt                                                    
2000-01-04 -0.084310 -0.083217 NaN  NaN  NaN -0.039106
2000-01-05  0.014634 -0.148741 NaN  NaN  NaN  0.001789
2000-01-06 -0.086538 -0.060036 NaN  NaN  NaN -0.016071
2000-01-07  0.047368  0.061010 NaN  NaN  NaN  0.058076
2000-01-10 -0.017588 -0.005391 NaN  NaN  NaN  0.003431

## Do a rolling regression on the dataframe with prepared data

In [17]:
ra = Reg(pct_df)
ma = ra.modelCols( [ idx["Pct", "SPY"]])
ma

beta_df = ra.rollingModelAll( *ma, #idx["Pct", "AAPL"],
                 pd.to_datetime("01/01/2000",infer_datetime_format=True),
                 # pd.to_datetime("04/14/2000", infer_datetime_format=True),
                 pd.to_datetime("12/29/2017", infer_datetime_format=True),
                 timedelta(weeks=4)
            )
beta_df.tail()

([('Pct', 'SPY')],
 [('Pct', 'AAPL'),
  ('Pct', 'AMZN'),
  ('Pct', 'FB'),
  ('Pct', 'GOOG'),
  ('Pct', 'NFLX')])

IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AAPL'), cols [('Pct', 'SPY'), ('Pct', 'AAPL')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AMZN'), cols [('Pct', 'SPY'), ('Pct', 'AMZN')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'FB'), cols [('Pct', 'SPY'), ('Pct', 'FB')]
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the follo

Beta 0                                            Beta 1  \
                AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                       
2017-09-08 -0.000272 -0.000858  0.000231  0.000056  0.000910  1.285798   
2017-10-06 -0.005136 -0.001802 -0.004168  0.001119  0.000013  2.281003   
2017-11-03  0.003963  0.001401 -0.000019  0.000439 -0.000820  1.733427   
2017-12-01 -0.001837  0.001098 -0.002488 -0.002125 -0.006099  1.339091   
2017-12-29 -0.001793 -0.000217 -0.000183  0.001072  0.000821  1.699265   

                                                    
                AMZN        FB      GOOG      NFLX  
Dt                                                  
2017-09-08  1.464521  1.288074  1.239199  1.292776  
2017-10-06  1.691252  2.566470  0.923909  3.312801  
2017-11-03  6.054137  2.509251  2.880951  1.734489  
2017-12-01  1.082630  1.164516  0.837844  2.177337  
2017-12-29  0.767625  0.859952  1.081924  0.908602

## Append the rolling betas to the prepared data

In [20]:
concatTrans = DataFrameConcat( [ pct_df, beta_df ])
ret_and_beta_df = concatTrans.fit_transform(pd.DataFrame())
ret_and_beta_df.loc[:"2017-12-29",:].tail()
ret_and_beta_df.shape

Beta 0                                            Beta 1  \
                AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                       
2017-12-22       NaN       NaN       NaN       NaN       NaN       NaN   
2017-12-26       NaN       NaN       NaN       NaN       NaN       NaN   
2017-12-27       NaN       NaN       NaN       NaN       NaN       NaN   
2017-12-28       NaN       NaN       NaN       NaN       NaN       NaN   
2017-12-29 -0.001793 -0.000217 -0.000183  0.001072  0.000821  1.699265   

                                                         Pct            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2017-12-22       NaN       NaN       NaN       NaN  0.000000 -0.005448   
2017-12-26       NaN       NaN       NaN       NaN -0.025370  0.007190   
2017-12-27       NaN       NaN       NaN       NaN  0.000176  0.004674   
2017-12-28       NaN       NaN       NaN       NaN  0.002814  0.003248   
2017-12-29  0.767625  0.859952  1.081924  0.908602 -0.010814 -0.014021   

                                                    
                  FB      GOOG      NFLX       SPY  
Dt                                                  
2017-12-22 -0.001409 -0.003300  0.006998 -0.000262  
2017-12-26 -0.006828 -0.003188 -0.011477 -0.001196  
2017-12-27  0.009262 -0.006974 -0.008095  0.000487  
2017-12-28  0.001689 -0.001172  0.034740  0.002057  
2017-12-29 -0.008206 -0.001660 -0.003892 -0.003771

(4569, 16)

In [12]:
gd.save_data(pct_df, "ret_df.pkl")
gd.save_data(beta_df, "beta_df.pkl")
gd.save_data(ret_and_beta_df, "ret_and_beta_df.pkl")

## Prepare for Return Attribution

### Find the attributes with the sensitivities

In [23]:
rab = RegAttr(ret_and_beta_df)
rab.setSens(beta_df)
betaAttrs = rab.sensAttrs('^Beta \d+$')
betaAttrs

['Beta 0', 'Beta 1']

## Roll the betas forward

In [46]:
beta_r_pl = make_pipeline( GenSelectAttrsTransformer(betaAttrs),
                            ShiftTransformer(1),
                            FillNullTransformer(method="ffill"),
                            GenRenameAttrsTransformer(lambda col: col + ' rolled fwd', level=0)
                         )
beta_rolled_df = beta_r_pl.fit_transform(ret_and_beta_df)
beta_rolled_df.tail()

transform: pandas version <= 0.20.


Beta 0 rolled fwd                                          \
                        AAPL      AMZN        FB      GOOG      NFLX   
Dt                                                                     
2018-02-13         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-14         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-15         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-16         -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-20         -0.001793 -0.000217 -0.000183  0.001072  0.000821   

           Beta 1 rolled fwd                                          
                        AAPL      AMZN        FB      GOOG      NFLX  
Dt                                                                    
2018-02-13          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-14          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-15          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-16          1.699265  0.767625  0.859952  1.081924  0.908602  
2018-02-20          1.699265  0.767625  0.859952  1.081924  0.908602

### Append the rolled betas to the regression results

In [48]:
ret_and_rolled_beta_pl = DataFrameConcat( [ ret_and_beta_df, beta_rolled_df])
ret_and_rolled_beta_df = ret_and_rolled_beta_pl.fit_transform( pd.DataFrame() )
ret_and_rolled_beta_df.tail()

Beta 0                    Beta 0 rolled fwd                      \
             AAPL AMZN  FB GOOG NFLX              AAPL      AMZN        FB   
Dt                                                                           
2018-02-13    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-14    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-15    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-16    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   
2018-02-20    NaN  NaN NaN  NaN  NaN         -0.001793 -0.000217 -0.000183   

                                  ...    Beta 1 rolled fwd            \
                GOOG      NFLX    ...                 AMZN        FB   
Dt                                ...                                  
2018-02-13  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-14  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-15  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-16  0.001072  0.000821    ...             0.767625  0.859952   
2018-02-20  0.001072  0.000821    ...             0.767625  0.859952   

                                     Pct                                \
                GOOG      NFLX      AAPL      AMZN        FB      GOOG   
Dt                                                                       
2018-02-13  1.081924  0.908602  0.010018  0.020401 -0.018480  0.000152   
2018-02-14  1.081924  0.908602  0.018437  0.025832  0.036789  0.016728   
2018-02-15  1.081924  0.908602  0.033578  0.007381  0.002451  0.018529   
2018-02-16  1.081924  0.908602 -0.003237 -0.008941 -0.014448  0.004846   
2018-02-20  1.081924  0.908602 -0.003364  0.013571 -0.007612  0.006997   

                                
                NFLX       SPY  
Dt                              
2018-02-13  0.001240  0.002487  
2018-02-14  0.029930  0.013496  
2018-02-15  0.053647  0.012760  
2018-02-16 -0.006244  0.000293  
2018-02-20  0.000108 -0.006115  

[5 rows x 26 columns]

In [29]:
reg = Reg(ret_and_rolled_beta_df)

### Find the columns for: 
#### independent variables
#### dependent variables
#### sensitivities

In [49]:
indCols = [ ("Pct", "1"), ("Pct", "SPY")]
indCols

rab2 = RegAttr(ret_and_rolled_beta_df)
rab2.setSens(beta_rolled_df)
sensAttrs = rab2.sensAttrs('^Beta \d+ rolled fwd$')
sensAttrs

depTickers = rab2.depTickersFromSensAttrs(sensAttrs )
depTickers
depCols = [ ("Pct", t) for t in depTickers ]
depCols



[('Pct', '1'), ('Pct', 'SPY')]

['Beta 0 rolled fwd', 'Beta 1 rolled fwd']

['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX']

[('Pct', 'AAPL'),
 ('Pct', 'AMZN'),
 ('Pct', 'FB'),
 ('Pct', 'GOOG'),
 ('Pct', 'NFLX')]

## Add constant (for interecept return) column

In [50]:
rab2.addConst(("Pct", "1"), 1)

addConst


### Perform the return attribution

In [51]:
retAttr_df =rab2.retAttrib(
            indCols,
            depCols, 
            sensAttrs)


In [52]:
retAttr_df.loc[:"2018-02-07",:].tail()

Contrib from 1                                          \
                     AAPL      AMZN        FB      GOOG      NFLX   
Dt                                                                  
2018-02-01      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-02      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-05      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-06      -0.001793 -0.000217 -0.000183  0.001072  0.000821   
2018-02-07      -0.001793 -0.000217 -0.000183  0.001072  0.000821   

           Contrib from SPY                                         Predicted  \
                       AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                              
2018-02-01        -0.001929 -0.000871 -0.000976 -0.001228 -0.001031 -0.003722   
2018-02-02        -0.036993 -0.016711 -0.018721 -0.023553 -0.019780 -0.038785   
2018-02-05        -0.071068 -0.032104 -0.035965 -0.045249 -0.038000 -0.072860   
2018-02-06         0.033479  0.015124  0.016943  0.021316  0.017901  0.031687   
2018-02-07        -0.009218 -0.004164 -0.004665 -0.005869 -0.004929 -0.011011   

                                                       Error            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2018-02-01 -0.001088 -0.001160 -0.000156 -0.000210  0.005812 -0.040879   
2018-02-02 -0.016928 -0.018904 -0.022481 -0.018959 -0.004605  0.045669   
2018-02-05 -0.032321 -0.036149 -0.044177 -0.037179  0.047876  0.004383   
2018-02-06  0.014907  0.016760  0.022389  0.018723  0.010105  0.023108   
2018-02-07 -0.004381 -0.004848 -0.004797 -0.004108 -0.010396 -0.013680   

                                          
                  FB      GOOG      NFLX  
Dt                                        
2018-02-01  0.034334 -0.001759 -0.019139  
2018-02-02  0.004352 -0.025305  0.027862  
2018-02-05 -0.011255 -0.006278 -0.012068  
2018-02-06  0.005584  0.001101  0.026349  
2018-02-07 -0.022835 -0.024835 -0.000258

In [37]:
gd.save_data(retAttr_df, "retattr_df.pkl")